In [ ]:
! pip install --upgrade category_encoders
! pip install interactiontransformer
! pip install scikit-learn==0.24
! pip install shap==0.39.0
! pip install transformers

In [ ]:
import pandas as pd
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
# from shapash.explainer.smart_explainer import SmartExplainer
# from eli5 import show_prediction
from sklearn.inspection import plot_partial_dependence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.ensemble import BalancedRandomForestClassifier
# import texthero as hero
# from texthero import preprocessing
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# import Lib
import re # For regular expressions
import numpy as np
import time
from tqdm import tqdm
import functools
import operator

# import scattertext as st
import matplotlib.pyplot as plt
import plotly.express as px

# sklearn imports
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.manifold import TSNE #T-Distributed Stochastic Neighbor Embedding


import spacy  
nlp = spacy.load('en_core_web_sm')
  
import nltk
import nltk.data
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
 
from gensim.models import word2vec


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import shap
import transformers

from IPython.display import clear_output

In [ ]:
class handle_data:
  def __init__(self, url):
    self.return_clf= True
    self.return_split= False
    self.df= pd.read_csv(url)
    # self.df= self.df.iloc[0:1000]
  
  def get_data(self):
    return self.df

  def split(self, target_col, size):
    y_df=self.df[target_col].to_frame()
    X_df=self.df[df.columns.difference([target_col])]
    categorical_features = [col for col in X_df.columns if X_df[col].dtype == 'object']

    encoder = OrdinalEncoder(
        cols=categorical_features,
        handle_unknown='ignore',
        return_df=True).fit(X_df)

    X_df=encoder.transform(X_df)
    return train_test_split(X_df, y_df, train_size=size, random_state=1), encoder


  def chooseModel(self, model_name, train_x, train_y):
    if (model_name == 'RF'):
      return RandomForestClassifier(max_depth=2, random_state=0).fit(train_x,train_y.values.ravel())
    elif (model_name=='GBM'):
      return LGBMClassifier(n_estimators=100, boosting_type='dart').fit(train_x,train_y.values.ravel())

  def Profile(self):
    profile = ProfileReport(self.df)
    return profile.to_widgets()
    # profile = ProfileReport(df=temp_df)
    # profile.to_file(output_file="output_min.html")

In [ ]:
data_handeler= handle_data('https://bitbucket.org/hari_narayanan_123/dataforexplainability/raw/5425adfe0e6b7dea984bf7adfd8dc589cd40d3b5/Training_datad9b6ebc.csv')

In [ ]:
df= data_handeler.get_data()

In [ ]:
split, encoder= data_handeler.split('Score',  size= 0.7)

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead


In [ ]:
Xtrain , Xtest, ytrain, ytest= split

In [ ]:
classifier= data_handeler.chooseModel("GBM", Xtrain, ytrain)

In [ ]:
y_pred= pd.DataFrame(classifier.predict(Xtest),columns=['pred'],index=Xtest.index)

##Shap Interpretability 

In [ ]:
class shap_visuals:
  def __init__(self, data, model):
    self.model = model
    self.data = data
    

  def forcePlot(self):
    lim=1000
    sample_data= self.data[:lim]
    explainer = shap.TreeExplainer(self.model)

    # calculate shap values. This is what we will plot.
    shap_values = explainer.shap_values(sample_data)
    return shap.force_plot(explainer.expected_value[0], shap_values[0], features= sample_data)

  def plot(self, idx):
    lim= 1000
    sample_data= self.data.iloc[:lim]
    explainer = shap.TreeExplainer(self.model)
    shap_values = explainer.shap_values(sample_data)
    for which_class in range(0,5):
      print(self.model.classes_[which_class])
      display(shap.force_plot(explainer.expected_value[which_class], shap_values[which_class][idx], np.array(sample_data.iloc[idx, :]).reshape(1, 9), link='logit'))

  def polarity(self, col):
    model = transformers.pipeline('sentiment-analysis', return_all_scores=True)

    # explain the model on two sample inputs
    explainer = shap.Explainer(model) 
    a= list(df[col])
    shap_values = explainer([a[0]])
    return shap.plots.text(shap_values[0, :, "POSITIVE"])



In [ ]:
sv= shap_visuals(Xtrain, classifier)

In [ ]:
shap.initjs()
sv.forcePlot()

In [ ]:
shap.initjs()
sv.plot(0)

1


2


3


4


5


In [ ]:
sv.polarity('Text')